In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

In [5]:
CLIENT_ID = '5EWKJF14LFOBC2HVPZZJIC0EGS1OWJS3ZRJO1QQEGOPYKO5V' # your Foursquare ID
CLIENT_SECRET = 'RV0JB2GJ1X23GLIALFR5UIKBU2LSEP4EXFSBKFJDG4H44T4J' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5EWKJF14LFOBC2HVPZZJIC0EGS1OWJS3ZRJO1QQEGOPYKO5V
CLIENT_SECRET:RV0JB2GJ1X23GLIALFR5UIKBU2LSEP4EXFSBKFJDG4H44T4J


In [6]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Da Nang",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)


In [7]:
d = data.json()["response"]
d.keys()


dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [8]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]


('city', 'Da Nang', 'Da Nang')

In [9]:
d["suggestedBounds"], d["totalResults"]


({'ne': {'lat': 16.076950003892705, 'lng': 108.22982675100442},
  'sw': {'lat': 16.059727142671775, 'lng': 108.21333222890732}},
 40)

In [10]:
d["geocode"]


{'what': '',
 'where': 'da nang',
 'center': {'lat': 16.06778, 'lng': 108.22083},
 'displayString': 'Da Nang, Thành Phố Đà Nẵng, Vietnam',
 'cc': 'VN',
 'geometry': {'bounds': {'ne': {'lat': 16.11072, 'lng': 108.276871},
   'sw': {'lat': 15.982205, 'lng': 108.141861}}},
 'slug': 'turan-vietnam',
 'longId': '72057594039511928'}

In [11]:
d["groups"][0].keys()


dict_keys(['type', 'name', 'items'])

In [12]:
d["groups"][0]["type"], d["groups"][0]["name"]


('Recommended Places', 'recommended')

In [13]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]


number of items: 40


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5a26a41a31ac6c676705e94c',
  'name': 'cộng cà phê',
  'location': {'address': '39',
   'crossStreet': 'Nguyễn Thái Học',
   'lat': 16.068062879455486,
   'lng': 108.22351222071603,
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.068062879455486,
     'lng': 108.22351222071603}],
   'postalCode': '551105',
   'cc': 'VN',
   'city': 'Đà Nẵng',
   'state': 'Thành Phố Đà Nẵng',
   'country': 'Việt Nam',
   'formattedAddress': ['39 (Nguyễn Thái Học)',
    'Đà Nẵng',
    'Thành Phố Đà Nẵng',
    'Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 're

In [16]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4d4361d01928a35daba1ad70',
  'name': 'Phố Xưa',
  'location': {'address': '17 Phan Đình Phùng',
   'lat': 16.070069923328905,
   'lng': 108.2234060288238,
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.070069923328905,
     'lng': 108.2234060288238}],
   'cc': 'VN',
   'city': 'Hải Châu',
   'state': 'Thành Phố Đà Nẵng',
   'country': 'Việt Nam',
   'formattedAddress': ['17 Phan Đình Phùng',
    'Hải Châu',
    'Thành Phố Đà Nẵng',
    'Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-4d4361d01928a35daba1ad70-1'}

In [18]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()


{'address': '39', 'crossStreet': 'Nguyễn Thái Học', 'lat': 16.068062879455486, 'lng': 108.22351222071603, 'labeledLatLngs': [{'label': 'display', 'lat': 16.068062879455486, 'lng': 108.22351222071603}], 'postalCode': '551105', 'cc': 'VN', 'city': 'Đà Nẵng', 'state': 'Thành Phố Đà Nẵng', 'country': 'Việt Nam', 'formattedAddress': ['39 (Nguyễn Thái Học)', 'Đà Nẵng', 'Thành Phố Đà Nẵng', 'Việt Nam']}
{'address': '17 Phan Đình Phùng', 'lat': 16.070069923328905, 'lng': 108.2234060288238, 'labeledLatLngs': [{'label': 'display', 'lat': 16.070069923328905, 'lng': 108.2234060288238}], 'cc': 'VN', 'city': 'Hải Châu', 'state': 'Thành Phố Đà Nẵng', 'country': 'Việt Nam', 'formattedAddress': ['17 Phan Đình Phùng', 'Hải Châu', 'Thành Phố Đà Nẵng', 'Việt Nam']}
{'address': '37-39 Pasteur', 'lat': 16.069698707615007, 'lng': 108.21804495108044, 'labeledLatLngs': [{'label': 'display', 'lat': 16.069698707615007, 'lng': 108.21804495108044}], 'cc': 'VN', 'city': 'Đà Nẵng', 'state': 'Thành Phố Đà Nẵng', 'cou

AttributeError: 'NoneType' object has no attribute 'items'

                        uid                  name    shortname address  \
0  5a26a41a31ac6c676705e94c           cộng cà phê  Coffee Shop           
1  5627c637498e21ca8f40f4be  Leaf Drinks and More  Coffee Shop           
2  55ff7cf0498e203b2932d5fd     Namunamu Cafe 180         Café           
3  4c25fdd8f1272d7f33ac85c5      Highlands Coffee         Café           
4  51bd1671498e58791c2ea92d        Freeman Coffee  Coffee Shop           

  postalcode        lat         lng  
0     551105  16.068063  108.223512  
1     550000  16.067020  108.224790  
2     550000  16.066268  108.224496  
3      60999  16.069946  108.224961  
4     550000  16.070851  108.221230  